In [ ]:
# Importação de bibliotecas
import pandas as pd
import numpy as np

In [ ]:
# Variáveis utilizadas
dataset_name = 'risco_erosao_pastagem'

raiz = '/home/eduardo/repositorios/IS_Agro/databases'

In [ ]:
# Tabelas necessárias para o desenvolvimento
df_ilpf = pd.read_parquet(f'{raiz}/prata_iLPF.parquet').reset_index()
df_pastagem = pd.read_parquet(f'{raiz}/ouro_is_agro_area_agropecuaria_pastagem.parquet')
df_vpastagem = pd.read_parquet(f'{raiz}/prata_lapig_pastagens_degradacao_uf.parquet').reset_index()
df_lapig = pd.read_parquet(f'{raiz}/prata_lapig_pastagem_uf.parquet').reset_index()


### Ouro

In [ ]:
df_novo = (
    df_pastagem
    .filter(['geocodigo', 'data'])
    .assign(**{
        'Não degradada': '',
        '40% severa': '',
        '60% severa': ''
    })
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='classe'
    )
    .drop(columns='value')
)

In [ ]:
df_deg_pastagem = (
    pd.concat([
        df_novo,
        df_vpastagem
    ])
    .merge(
        df_pastagem,
        how='right',
        on=['geocodigo', 'data']
    )
    .merge(
        df_lapig,
        how='left',
        on=['geocodigo', 'data'],
        suffixes=['_vigor', '_total_lapig']
    )
    .merge(
        df_ilpf,
        how='left',
        on=['geocodigo', 'data'],
        suffixes=['_isagro', '_ilpf']
    )
    .assign(
        valor_ilpf = lambda n: n.valor_ilpf.fillna(0), # Substituição de valor nulo por zero para futuro uso nos cálculos
        area_vigor=lambda df: df.groupby(['geocodigo', 'data'], group_keys=False).apply( # Agrupamento como index das duas colunas, com exclusão das mesmas na sequência da programação 
            lambda gp: gp.assign(
                area_vigor=np.where(
                    gp['classe'].isin(['40% severa', '60% severa']), # Verificação em classe se é "40% severa" ou "60% severa"
                    gp.loc[gp['classe'] == 'Severa', 'area_vigor'].values[0] if not gp.loc[gp['classe'] == 'Severa', 'area_vigor'].empty else np.nan, # Extração do valor de area_vigor onde classe é "Severa", caso não, imputar valor nulo
                    gp['area_vigor']
                )
            )
        )['area_vigor'],
        valor = lambda n: np.where(n.classe == 'Não degradada', n.valor_ilpf,
                                   np.where(n.classe == 'Ausente', n.area_vigor,
                                            np.where(n.classe == 'Intermediário', n.area_vigor - (n.valor_ilpf/3),
                                                     np.where(n.classe == '40% severa', ((0.4 * n.area_vigor) - n.valor_ilpf/3),
                                                              np.where(n.classe == '60% severa', ((0.6 * n.area_vigor) - n.valor_ilpf/3),
                                                                       pd.NA)                                                                                                              
                                                            )
                                                    )                                       
                                        )
                                )                                                 
    )
    .convert_dtypes()
    .query('classe != "Severa"')
    .drop(columns=['area_vigor', 'valor_isagro', 'area_total_lapig', 'valor_ilpf'])
    .assign(valor = lambda ha: round(ha.valor/10000, 2))
    .replace({
        'Não degradada': 'Pastagem Não Degradada',
        'Ausente': 'Pastagem com Degradação Leve',
        'Intermediário': 'Pastagem com Degradação Leve/Moderada',
        '40% severa': 'Pastagem com Degradação Moderada',
        '60% severa': 'Pastagem com Degradação Severa'
    })
    .assign(
        risco_erosivo = lambda df: np.where(df.classe == 'Pastagem Não Degradada', 4.5,
                                        np.where(df.classe == 'Pastagem com Degradação Leve', 4.5,
                                                 np.where(df.classe == 'Pastagem com Degradação Leve/Moderada', 22.4,
                                                          np.where(df.classe == 'Pastagem com Degradação Moderada', 30.0,
                                                                   np.where(df.classe == 'Pastagem com Degradação Severa', 35.0,
                                                                            pd.NA)))))
    )
)

In [ ]:
df_ouro = (
    df_deg_pastagem
    .rename(columns={
        'geocodigo': 'state',
        'valor': 'value',
        'data': 'date',
        'classe': 'label',
        'risco_erosivo': 'sub_label'
        })
    .assign(country = 76)
    .filter(['country', 'state', 'date', 'label', 'sub_label', 'value'])
)

df_ouro.to_csv(f'{raiz}/ouro_{dataset_name}.csv', index=False)